The MNIST dataset contains tens of thousands of 28px by 28px images of hand-written digits. Handwriting varies from person to person, so building a generalizable machine learning model that can very accurately identify hand written digits, which is the objective here, can be a bit of a challenge. 

On top of that, there is an issue with the sheer number of input variables: 28 \* 28 = 784 input pixels. If a fully-connected neural network with an equal number of nodes in its first layer is utlilized to identify the digits, then there would be 784 inputs * 784 nodes = 614,656 weights to learn in just the first layer alone. This makes utilizing regular neural networks computationally expensive.

Fortunately, there is an alternative: convolutional neural networks. Convolutional neural networks reduce the number of weights for a fully connected neural network by convolution and by pooling. Fairly good explanations of the two processes can be found in these two YouTube videos: https://youtu.be/NVH8EYPHi30 and https://youtu.be/umGJ30-15_A. The rough idea is that this by convolution and pooling, patches of the image are summarized. What goes into a summary is something the is learned by the network itself. By summarizing, the image is, in a way, "compressed." The "compressed" image is then fed into some classification algorithm (typically a regular neural network) that does the actual classification. 

There are many kinds of convolutional neural networks, especially those geared towards accurately identifying MNSIT digits. However, the classic version will be utilized here given that this is my first serious look at machine learning for identifying things in images. The network in this notebook will be based off of the ones found here: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py and https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html. 

In [1]:
import pandas
import numpy as np

train = pandas.read_csv('train.csv')
test = pandas.read_csv('test.csv')

In [2]:
train.sample(n=5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1386,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21322,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17842,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27557,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31601,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Just out of curiosity, I wonder what the handwritten digits look like.

In [3]:
# Helped: https://stackoverflow.com/questions/2659312/how-do-i-convert-a-numpy-array-to-and-display-an-image
from PIL import Image

def display_mnist_digit(digit_array):
    """
    Display the digit in the MNIST data set.
    
    :param digit_array: 1D numpy array of the data
    """
    image_array = digit_array.astype(np.uint8)
    img = Image.fromarray(image_array.reshape(28, 28))

    display(img)
    
num_image_to_display = 10
for idx in range(222, 222+num_image_to_display):
    display_mnist_digit(train.drop('label', axis=1).iloc[idx].values)

The plan is to learn how to use a convolution neural network (CNN); however, I first want to use a regular, fully connected neural network (a multilayer perceptron). Hopefully it will give us a baseline by which the CNN can be judged. 

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x_train, x_test, y_train, y_test = train_test_split(train.drop('label', axis=1),
                                                    train['label'], 
                                                    test_size=0.25)

mlpc = MLPClassifier(hidden_layer_sizes=[1400]) #1400 nodes

mlpc.fit(x_train, y_train)

accuracy_score(y_test, mlpc.predict(x_test))

0.9594444444444444

That's actually not an absolutely terrible baseline. If this accuracy score is accurate, then this model would place at about 2153rd place out of 2646 teams: ahead of about 20% of the all models. Again, though, this is just a baseline; the goal is to build a CNN to classify images.

The images are currently represented as 1D arrays but a CNN, typically, expects a 2D array. The reason for this has to do with the convolution part in the covolutional neural network. The actual process of convolution is covered well in these two YouTube videos: https://youtu.be/NVH8EYPHi30 and https://youtu.be/umGJ30-15_A. The basic idea is that convolution takes advantage of spatial relationships (pixels near each other are probably related) and sort of summarizes patches of an image.

Let's convert the images into 2D arrays. This will, inevitably, take up more memory: at least 1 bytes per 8-bit int * 784 pixels per image * 42,000 images = 32,928,000 or 32.928MB. The alternative, however, is to do this conversion as part of a pipeline, which, if the pipeline is run more than once, would not be efficient speed-wise, only memory-wise. 

The 32.928MB number, at least on my machine with 4GB of RAM, is not too large. For other data sets with hundreds of thousands of images that each have millions of pixels (real world data sets), converting everything at once would be more difficult. For example, a data set with 100,000 black-and-white images each with 1280 * 720 pixels (a 720p image) would be at least 92,160,000,000 bytes or 92.16 GB. 

Therefore, by converting to and then saving 2D representations of the mnist digit images for the CNN once at the start, we can tune the CNN later without having to redo this conversion and without trading-off all of our RAM/memory.

In [5]:
# 4D array because 
# 1st dim = image index
# 2nd and 3rd dims = width and height (column and row coordinates of a pixel in an image)
# 4th dim = channel(s) (RGB or luminance if only gray scale)
# shape: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
x = train.drop('label', axis=1).values.reshape(train.shape[0], 28, 28, 1)
y = train['label']

# Want to be able to use categorical cross entropy as a loss
# measurement, which requires dummy encoding train columns
y_binary = pandas.get_dummies(pandas.DataFrame([str(label) for label in y],
                                               columns=['label']))

# put columns in sorted order (0 at lowest idx; 9 at highest idx)
y_binary = y_binary[ sorted(y_binary.columns) ]

print('Successful dummy encoding?: {}.'.format(y_binary.shape[1]==10))
print(y_binary.columns)

Successful dummy encoding?: True.
Index(['label_0', 'label_1', 'label_2', 'label_3', 'label_4', 'label_5',
       'label_6', 'label_7', 'label_8', 'label_9'],
      dtype='object')


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x,
                                                  y_binary,
                                                  test_size=0.25)

The standard convolutional neural network has two parts. The first part contains a convolutional layer, then an activation layer (like the rectified linear unit (ReLU) that is max{0, x}), and then a pooling layer (like a max pooling layer that keeps only the max value in a patch of convoluted pixels) to reduce dimensionality. This set, convolution then activation then pooling, can be repeated multiple times.

In the second section, the final pooling outputs are fed into a fully connected network that actually makes the classification. The first part can be though of as extracting spatial features while the second part can be though of as taking those features and making some classification. This model will draw upon: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py.

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()

# PART 1
# CONVOLUTION LAYER 1
model.add(Conv2D(96, 
                 kernel_size=(4,4),  # convolution window size: hxw
                 strides = (1,1), # how far move window after each convolution
                 input_shape=(28, 28, 1), # image w, image h, num channels/colors
                 data_format="channels_last") # channel=colors (RGB or luminance)
         )
model.add(Activation('relu')) # rectified linear unit
# model.add(MaxPooling2D(pool_size=(2,2))) # 2x2 patches become 1x1's 

# CONVOLUTION LAYER 2
model.add(Conv2D(160, kernel_size=(3,3), strides=(1,1)))
model.add(Activation('relu')) # rectified linear unit
model.add(MaxPooling2D(pool_size=(2,2))) # 2x2 patches become 1x1's 
model.add(Dropout(0.28)) # rand set frac of inputs to 0 - improve generalizability

# CONVOLUTION LAYER 3
model.add(Conv2D(96, kernel_size=(2,2), strides=(1,1)))
model.add(Activation('relu'))


# PART 2
# FULLY CONNECTED
# now take output from convolution nn and feed it into  fully connected nn
model.add(Flatten()) # not lose spatial info since "encoded" via convolution?
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.28)) # rand set frac of inputs to 0 - improve generalizability

model.add(Dense(10)) # 10 output nodes, 1 for each number
model.add(Activation('sigmoid'))


# Be careful with loss and metrics; accuracy =/=> categorical accuracy: 
# https://stackoverflow.com/questions/41327601/why-is-binary-crossentropy-more-accurate-than-categorical-crossentropy-for-multi
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'],)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'],)

model.fit(x_train, y_train, epochs=15, batch_size=256)


Using TensorFlow backend.


Epoch 1/15
39900/39900 [==============================] - 434s 11ms/step - loss: 1.4987 - categorical_accuracy: 0.1945
Epoch 2/15
39900/39900 [==============================] - 481s 12ms/step - loss: 1.1106 - categorical_accuracy: 0.3434
Epoch 3/15
39900/39900 [==============================] - 490s 12ms/step - loss: 0.0294 - categorical_accuracy: 0.9587
Epoch 4/15
39900/39900 [==============================] - 477s 12ms/step - loss: 0.0148 - categorical_accuracy: 0.9801
Epoch 5/15
39900/39900 [==============================] - 495s 12ms/step - loss: 0.0109 - categorical_accuracy: 0.9852
Epoch 6/15
39900/39900 [==============================] - 467s 12ms/step - loss: 0.0085 - categorical_accuracy: 0.9895
Epoch 7/15
39900/39900 [==============================] - 471s 12ms/step - loss: 0.0072 - categorical_accuracy: 0.9907
Epoch 8/15
39900/39900 [==============================] - 483s 12ms/step - loss: 0.0061 - categorical_accuracy: 0.9927
Epoch 9/15
39900/39900 [========================

#### Reasoning behind some of the modeling decisions

+ Square convolution window: the image itself is square. Non-square windows were tried since numbers are more rectangular in shape, but square windows still performed better.

+ binary_cross_entropy & categorical accuracy: easier to treat this as a multiclass classification problem instead of a regression problem given small number of discrete digits.

+ adam: this optimizer is fairly fast, requires little to no tuning, and in my limited experience, achieves results comparable to those of other optimizers (SGD, nadam, etc.).

+ dropouts: dropout serves as a sort of regularization term. It seems counterintuitive to want to randomly set some activiation values to 0, but it is a little like adding randomness for which the model has to account. Overall, it does seem to help reduce overfitting.

+ commented out 1st pooling layer: improved accuracy, which makes sense pooling is more of a dimensionality reduction tool rather than a feature extraction technique like convolution.


This is very promising! Let's see how it performs on some testing data.

*Note, the above model is not the first one tried; it has been tuned.

In [79]:
# Argmax idea from: https://github.com/keras-team/keras/issues/2524

def categorical_accuracy_calc(y_actual, y_predicted):
    correct_classifications = 0 
    
    for actual, predicted in zip(y_actual, y_predicted):
        correct_classifications += (np.argmax(actual) == np.argmax(predicted))
        
    return correct_classifications / float(len(y_predicted))

In [80]:
val_predictions = model.predict(x_val)
acc = categorical_accuracy_calc(y_val.values, val_predictions)
print('CNN is {0:.5f}% accurate on validation data.'.format(acc*100))


CNN is 97.14286% accurate on validation data.


Let's see what digits the CNN misclassified and what the images look like.

In [78]:
# Print the ones the CNN misclassified
misclassified_digit_indicies = []
for idx in range(0, len(val_predictions)):
    if np.argmax(y_val.values[idx]) != np.argmax(val_predictions[idx]):
        misclassified_digit_indicies.append(idx)

for idx in misclassified_digit_indicies:
    display_mnist_digit(x_val[idx])
    print('actual:{} \t\t predicted: {}'.format(np.argmax(y_val.values[idx]), np.argmax(val_predictions[idx])))

actual:9 		 predicted: 5


actual:6 		 predicted: 4


actual:1 		 predicted: 4


actual:2 		 predicted: 1


actual:4 		 predicted: 1


...

actual:5 		 predicted: 6


actual:8 		 predicted: 6


actual:1 		 predicted: 2


actual:3 		 predicted: 5


actual:5 		 predicted: 3


While searching around about how to reduce overfitting in a convolutional neural network and how to squeeze out more performance, I stumbled upon this wordpress: https://shaoanlu.wordpress.com/2017/05/29/sgd-all-which-one-is-the-best-optimizer-dogs-vs-cats-toy-experiment/. According to this person's summary of a journal article, adam optimizers do not build good generalizable models; they overfit the training data. The recommended approach is to, instead, do a combination of adam and stochastic gradient descenet (SGD).

Fortunately, Keras makes this rather simple according to: https://stackoverflow.com/questions/47626254/changing-optimizer-in-keras-during-training?rq=1. The model can be recompiled with a different optimizer without losing its weights that it already learned from training. The recompiled model can be fitted, and the new optimizer will be used, picking up where the previous compilation left off. I am interested to see if this will actually meaningfully improve performance, so let's try it.

In [11]:
from keras.optimizers import SGD

model_fine_tuned = model

model_fine_tuned.compile(loss='binary_crossentropy',
                         optimizer=SGD(lr=0.03, momentum=0.2),
                         metrics=['categorical_accuracy'],)

model_fine_tuned.fit(x_train, y_train, epochs=3, batch_size=256)

Epoch 1/3
39900/39900 [==============================] - 490s 12ms/step - loss: 0.0019 - categorical_accuracy: 0.9984
Epoch 2/3
39900/39900 [==============================] - 483s 12ms/step - loss: 0.0017 - categorical_accuracy: 0.9985
Epoch 3/3
39900/39900 [==============================] - 480s 12ms/step - loss: 0.0017 - categorical_accuracy: 0.9983


In [12]:
model.save('cnn_3_layers_submission.h5')

In [13]:
val_predictions = model_fine_tuned.predict(x_val)
acc = categorical_accuracy_calc(y_val.values, val_predictions)
print('CNN is {0:.5f}% accurate on validation data.'.format(acc*100))


CNN is 98.66667% accurate on validation data.


Wow! That's actually a meaningful improvement, at least for the Kaggle public leaderboards.

In [15]:
test = pandas.read_csv('test.csv')

result = pandas.DataFrame({'ImageId': [img_id for img_id in range(1, test.shape[0]+1)]})

result['Label'] = model_fine_tuned.predict_classes(test.values.reshape( test.shape[0], 28, 28, 1) )

print('writing to csv')
# no label column in test, so can just reshape values
result.to_csv('mnist_cnn_3_layers_25_dropout.csv', index=False, sep=',')
print('done!')

writing to csv
done!


At this point, there are other improvements that could be made to the model. More hidden layers could be added (6 hidden convolution layers seems to be popular), the digits could be deskewed (de-slanted, really), and the image could be elastic-distored (shrinking or stretching the image at random points) (see https://en.wikipedia.org/wiki/MNIST_database#Performance). All of those things, however, are essentially just copy and pasting other peoples' solutions from research papers and blogs. Submitting solutions that use those things feels dishonest, so this is where I will leave off. 

The final model scored was **98.985%** accurate on the Kaggle leaderboard. That puts the model at 1056th place out of 2658 teams (**top 40%**). For reference, humans on average supposedly correctly identify 97.5% to 98% of digits, so this model exceeds that level of accuracy.